In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data" , one_hot = True)
## 每个批次的大小
batch_size = 100

# 计算一共有多少批次
n_batch = mnist.train.num_examples  // batch_size

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
def inception_block(x,
                    output_channel_for_each_path,
                    name):
    
    with tf.variable_scope(name):
        conv1_1 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[0],
                                   (1, 1),
                                   strides = (1,1),
                                   padding = 'same',
                                   activation = tf.nn.relu,
                                   name = 'conv1_1')
        conv3_3 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[1],
                                   (3, 3),
                                   strides = (1,1),
                                   padding = 'same',
                                   activation = tf.nn.relu,
                                   name = 'conv3_3')
        conv5_5 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[2],
                                   (5, 5),
                                   strides = (1,1),
                                   padding = 'same',
                                   activation = tf.nn.relu,
                                   name = 'conv5_5')
        max_pooling = tf.layers.max_pooling2d(x,
                                              (2, 2),
                                              (2, 2),
                                              name = 'max_pooling')
    
    max_pooling_shape = max_pooling.get_shape().as_list()[1:]
    input_shape = x.get_shape().as_list()[1:]
    width_padding = (input_shape[0] - max_pooling_shape[0]) // 2
    height_padding = (input_shape[1] - max_pooling_shape[1]) // 2
    temp = 0
    print(conv5_5.get_shape().as_list()[3] - 4*width_padding)
    if conv5_5.get_shape().as_list()[3] - 4*width_padding == 4:
        temp = 1
    elif conv5_5.get_shape().as_list()[3] - 4*width_padding == -4:
        temp = -1
    padded_pooling = tf.pad(max_pooling,
                            [[0, 0],
                             [width_padding+temp, width_padding],
                             [height_padding+temp, height_padding],
                             [0, 0]])
    concat_layer = tf.concat(
        [conv1_1, conv3_3, conv5_5, padded_pooling],
        axis = 3)
    return concat_layer

In [3]:
# 定义placeholder
x = tf.placeholder(tf.float32 , [None , 784])
y = tf.placeholder(tf.float32 , [None , 10])
lr = tf.Variable(0.001,dtype=tf.float32)

# 改变x的格式转为4D的向量 [batch , in_height , in_width , in_channels]
x_image = tf.reshape(x , [-1,28,28,1])

# conv1: 神经元图， feature_map, 输出图像
conv1 = tf.layers.conv2d(x_image,
                         32, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv1')

pooling1 = tf.layers.max_pooling2d(conv1,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool1')

inception_2a = inception_block(pooling1,
                               [16, 16, 16],
                               name = 'inception_2a')
inception_2b = inception_block(inception_2a,
                               [16, 16, 16],
                               name = 'inception_2b')

pooling2 = tf.layers.max_pooling2d(inception_2b,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool2')

inception_3a = inception_block(pooling2,
                               [16, 16, 16],
                               name = 'inception_3a')
inception_3b = inception_block(inception_3a,
                               [16, 16, 16],
                               name = 'inception_3b')

pooling3 = tf.layers.max_pooling2d(inception_3b,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool3')

flatten = tf.layers.flatten(pooling3)

# 得到预测值
prediction = tf.layers.dense(flatten, 10)

# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y , logits=prediction))

# 使用梯度下降
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

# 结果存放在一个布尔型变量中
correct_prediction = tf.equal(tf.argmax(y,1) , tf.argmax(prediction,1))

# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
4
4
8
8
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
# 初始化变量
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(20):
        sess.run(tf.assign(lr,0.0005*(0.95**epoch)))
        for batch in range(n_batch):
            batch_xs , batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step , feed_dict={
                x:batch_xs,
                y:batch_ys
            })
        acc = sess.run(accuracy , feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels
        })
        print("Iter" + str(epoch) + ",Testing Accuracy : " + str(acc))

Iter0,Testing Accuracy : 0.9817
Iter1,Testing Accuracy : 0.9827
Iter2,Testing Accuracy : 0.9901
Iter3,Testing Accuracy : 0.9913
Iter4,Testing Accuracy : 0.989
Iter5,Testing Accuracy : 0.9912
Iter6,Testing Accuracy : 0.9915
Iter7,Testing Accuracy : 0.9915
Iter8,Testing Accuracy : 0.9929
Iter9,Testing Accuracy : 0.9933
Iter10,Testing Accuracy : 0.9932
Iter11,Testing Accuracy : 0.9928
Iter12,Testing Accuracy : 0.9924
Iter13,Testing Accuracy : 0.9922
Iter14,Testing Accuracy : 0.9941
Iter15,Testing Accuracy : 0.9924
Iter16,Testing Accuracy : 0.9935
Iter17,Testing Accuracy : 0.9932
Iter18,Testing Accuracy : 0.9942
Iter19,Testing Accuracy : 0.9923
